In [1]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np

from spacy.util import minibatch, compounding

# Abrindo e gerando Datasets

### Entidade logradouro

In [2]:
# Obtendo dados crus de endereço

dataset = pd.read_csv("201906AGENCIAS.CSV")
dset = dataset.iloc[:-2,4:10].values  # tipo nparray object
pd_dset = pd.DataFrame(dset) # caso queira visualizar

pd_dset['Join'] = pd_dset[pd_dset.columns[0:]].apply(
    lambda x: ';'.join(x.dropna().astype(str)),
    axis=1
)
pd_dset.head(5)

,0,1,2,3,4,5,Join
0,"R.GUILHERME MOREIRA,315","SUBLOJA,LOJA E 2.ANDAR ...",CENTRO,69005-300,MANAUS ...,AM,"R.GUILHERME MOREIRA,315 ;SUBLO..."
1,"AV.PRES.VARGAS,248",1.E 2.ANDARES ...,CAMPINA,66010-900,BELEM ...,PA,"AV.PRES.VARGAS,248 ;1.E 2..."
2,"R.QUINZE DE NOVEMBRO,195",...,CENTRO,11010-908,SANTOS ...,SP,"R.QUINZE DE NOVEMBRO,195 ; ..."
3,"PCA.DAS QUATRO JORNADAS,11",MEZANINO ...,CENTRO,28010-000,CAMPOS DOS GOYTACAZES ...,RJ,"PCA.DAS QUATRO JORNADAS,11 ;MEZAN..."
4,"SEXTA AVENIDA,600",SECRETARIA DA EDUCACAO-TERREO ...,CAB,41745-002,SALVADOR ...,BA,"SEXTA AVENIDA,600 ;SECRE..."


In [3]:
# Tratando endereço completo do DataFrame
dset = np.array(pd_dset)
end_lista = []

for i in range(len(dset)):
    str_raw = dset[i][6]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    str_tratada = re.sub(r'[;]{1,}', "; ", str_tratada) # Para complementos vazios, para não ter 2 ";"
    str_tratada = str_tratada.lower()
    
    end_lista.append(str_tratada)

end_lista[:3]

['r.guilherme moreira,315; subloja,loja e 2.andar; centro; 69005-300; manaus; am',
 'av.pres.vargas,248; 1.e 2.andares; campina; 66010-900; belem; pa',
 'r.quinze de novembro,195; centro; 11010-908; santos; sp']

In [4]:
# Lógica para extrair posição do logradouro:
# Dividir a String inteira por ";", pegar o len do primeiro split
# len de LOGRA é de 0 até len do primeiro split

print(end_lista[0])
split = end_lista[0].split(";")
print(split)
print(split[0])
print(len(split[0]))

r.guilherme moreira,315; subloja,loja e 2.andar; centro; 69005-300; manaus; am
['r.guilherme moreira,315', ' subloja,loja e 2.andar', ' centro', ' 69005-300', ' manaus', ' am']
r.guilherme moreira,315
23


In [5]:
# Extraindo iob com entidade apenas de logradouro

iob = []

for i in range(len(end_lista)):
    split = end_lista[i].split(";")
    len_logra = len(split[0])
    iob_dict = {"entities": [(0, len_logra-1,'LOGRA')]}
    end_lista[i] = end_lista[i].replace(";", " ")
    end_lista[i] = end_lista[i].replace(",", " ")
    end_lista[i] = re.sub(r'[ ]{2,}', " ", end_lista[i])

    
    tupla = (end_lista[i], iob_dict)
    iob.append(tupla)

FULL_DATA = iob

In [6]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])
print(FULL_DATA[5:15])

('r.guilherme moreira 315 subloja loja e 2.andar centro 69005-300 manaus am', {'entities': [(0, 22, 'LOGRA')]})
('av.pres.vargas 248 1.e 2.andares campina 66010-900 belem pa', {'entities': [(0, 17, 'LOGRA')]})
('r.quinze de novembro 195 centro 11010-908 santos sp', {'entities': [(0, 23, 'LOGRA')]})
[('av.rio branco 240 1.andar recife antigo 50030-310 recife pe', {'entities': [(0, 16, 'LOGRA')]}), ('av.santos dumont 2828 5.andar aldeota 60150-162 fortaleza ce', {'entities': [(0, 20, 'LOGRA')]}), ('pca.tiradentes 410 1.andar centro 80020-100 curitiba pr', {'entities': [(0, 17, 'LOGRA')]}), ('r.uruguai 185 5.andar centro 90010-901 porto alegre rs', {'entities': [(0, 12, 'LOGRA')]}), ('pca.1817 129 1.andar centro 58013-010 joao pessoa pb', {'entities': [(0, 11, 'LOGRA')]}), ('pca.odilon resende andrade 76 centro 37410-000 tres coracoes mg', {'entities': [(0, 28, 'LOGRA')]}), ('av fernandes lima 2591 terreo farol 57057-972 maceio al', {'entities': [(0, 22, 'LOGRA')]}), ('r.treze de junho 91

In [7]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

Treinamento: 19202 
Teste: 2134 
Total: 21336


In [8]:
for i in range(5):
    print(base_treinamento[i])

('rua fernando luzzatto 208 95320-000 nova prata rs', {'entities': [(0, 25, 'LOGRA')]})
('av.olinto meireles milionarios 30620-330 belo horizonte mg', {'entities': [(0, 17, 'LOGRA')]})
('av.maj.felizardo pinho pessoa 101 terreo centro 62300-000 vicosa do ceara ce', {'entities': [(0, 32, 'LOGRA')]})
('av.paschoal ardito 726 sao vito 13472-130 americana sp', {'entities': [(0, 21, 'LOGRA')]})
('rua do catete 117 parte gloria 22220-000 rio de janeiro rj', {'entities': [(0, 17, 'LOGRA')]})


# Carregando o modelo

In [14]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 100 # número de épocas
batch_size = 64

In [10]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [11]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [15]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, ' Losses', losses)        

Batch size:  64
Épocas:  100

1  Losses {'ner': 760.4642904412153}
2  Losses {'ner': 687.9467526326686}
3  Losses {'ner': 571.2996361747396}
4  Losses {'ner': 584.0007234295372}
5  Losses {'ner': 709.6346988221127}
6  Losses {'ner': 677.8147405720304}
7  Losses {'ner': 751.0372388633491}
8  Losses {'ner': 654.6342283573957}
9  Losses {'ner': 807.3264651178127}
10  Losses {'ner': 648.1604003574575}
11  Losses {'ner': 725.218250008531}
12  Losses {'ner': 710.8368854340899}
13  Losses {'ner': 694.51988705475}
14  Losses {'ner': 640.6430908711286}
15  Losses {'ner': 613.810349366108}
16  Losses {'ner': 655.73425533827}
17  Losses {'ner': 590.9776571386733}
18  Losses {'ner': 563.9133672648928}
19  Losses {'ner': 445.7783034450096}
20  Losses {'ner': 662.9166887719142}
21  Losses {'ner': 563.531455903086}
22  Losses {'ner': 450.82243060089115}
23  Losses {'ner': 477.7843401953881}
24  Losses {'ner': 502.37731827908556}
25  Losses {'ner': 484.23391088189686}
26  Losses {'ner': 534.8800630784

In [16]:
# test the trained model
for text, _ in base_teste:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print()

Entities [('av. mal rondon', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('mal', 'LOGRA', 1), ('rondon', 'LOGRA', 1), ('/', '', 2), ('rua', '', 2), ('princesa', '', 2), ('isabel', '', 2), ('centro', '', 2), ('78968-000', '', 2), ('pimenta', '', 2), ('bueno', '', 2), ('ro', '', 2)]

Entities [('r frei caneca 569', 'LOGRA')]
Tokens [('r', 'LOGRA', 3), ('frei', 'LOGRA', 1), ('caneca', 'LOGRA', 1), ('569', 'LOGRA', 1), ('bela', '', 2), ('vista', '', 2), ('01307-001', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Entities [('av. governador carlos de lima cavalcante num 4100', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('governador', 'LOGRA', 1), ('carlos', 'LOGRA', 1), ('de', 'LOGRA', 1), ('lima', 'LOGRA', 1), ('cavalcante', 'LOGRA', 1), ('num', 'LOGRA', 1), ('4100', 'LOGRA', 1), ('casa', '', 2), ('caiada', '', 2), ('53040-000', '', 2), ('olinda', '', 2), ('pe', '', 2)]

Entities [('av. paulista 1429', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA

Entities [('praca eleoterio g andrade 537', 'LOGRA')]
Tokens [('praca', 'LOGRA', 3), ('eleoterio', 'LOGRA', 1), ('g', 'LOGRA', 1), ('andrade', 'LOGRA', 1), ('537', 'LOGRA', 1), ('sala', '', 2), ('3', '', 2), ('centro', '', 2), ('87250-000', '', 2), ('peabiru', '', 2), ('pr', '', 2)]

Entities [('av. brigadeiro lima e silva 1685', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('brigadeiro', 'LOGRA', 1), ('lima', 'LOGRA', 1), ('e', 'LOGRA', 1), ('silva', 'LOGRA', 1), ('1685', 'LOGRA', 1), ('centro', '', 2), ('25070-235', '', 2), ('duque', '', 2), ('de', '', 2), ('caxias', '', 2), ('rj', '', 2)]

Entities [('praca costa pereira', 'LOGRA')]
Tokens [('praca', 'LOGRA', 3), ('costa', 'LOGRA', 1), ('pereira', 'LOGRA', 1), ('53', '', 2), ('-', '', 2), ('salas', '', 2), ('501/505', '', 2), ('(', '', 2), ('parte', '', 2), (')', '', 2), ('centro', '', 2), ('29010-080', '', 2), ('vitoria', '', 2), ('es', '', 2)]

Entities [('av japao', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('japao', 'LOG

Entities [('r do humaita', 'LOGRA')]
Tokens [('r', 'LOGRA', 3), ('do', 'LOGRA', 1), ('humaita', 'LOGRA', 1), ('humaita', '', 2), ('22261-005', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), ('rj', '', 2)]

Entities [('campus 1 - s/n', 'LOGRA')]
Tokens [('campus', 'LOGRA', 3), ('1', 'LOGRA', 1), ('-', 'LOGRA', 1), ('s', 'LOGRA', 1), ('/', 'LOGRA', 1), ('n', 'LOGRA', 1), ('cidade', '', 2), ('universitaria', '', 2), ('58051-970', '', 2), ('joao', '', 2), ('pessoa', '', 2), ('pb', '', 2)]

Entities [('tv.mal.deodoro da fonseca 22', 'LOGRA')]
Tokens [('tv.mal.deodoro', 'LOGRA', 3), ('da', 'LOGRA', 1), ('fonseca', 'LOGRA', 1), ('22', 'LOGRA', 1), ('centro', '', 2), ('49600-000', '', 2), ('nossa', '', 2), ('senhora', '', 2), ('das', '', 2), ('dores', '', 2), ('se', '', 2)]

Entities [('praca cel. pedro osorio 164', 'LOGRA')]
Tokens [('praca', 'LOGRA', 3), ('cel', 'LOGRA', 1), ('.', 'LOGRA', 1), ('pedro', 'LOGRA', 1), ('osorio', 'LOGRA', 1), ('164', 'LOGRA', 1), ('centro', '', 2), 

Entities [('rua edgar raimundo da costa 108', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('edgar', 'LOGRA', 1), ('raimundo', 'LOGRA', 1), ('da', 'LOGRA', 1), ('costa', 'LOGRA', 1), ('108', 'LOGRA', 1), ('centro', '', 2), ('16800-000', '', 2), ('mirandopolis', '', 2), ('sp', '', 2)]

Entities [('rua leopoldo freiberger 567', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('leopoldo', 'LOGRA', 1), ('freiberger', 'LOGRA', 1), ('567', 'LOGRA', 1), ('centro', '', 2), ('88160-000', '', 2), ('biguacu', '', 2), ('sc', '', 2)]

Entities [('av.general flores da cunha 1738', 'LOGRA')]
Tokens [('av.general', 'LOGRA', 3), ('flores', 'LOGRA', 1), ('da', 'LOGRA', 1), ('cunha', 'LOGRA', 1), ('1738', 'LOGRA', 1), ('vila', '', 2), ('city', '', 2), ('velha', '', 2), ('94910-003', '', 2), ('cachoeirinha', '', 2), ('rs', '', 2)]

Entities [('r.bernardo sayao 440', 'LOGRA')]
Tokens [('r.bernardo', 'LOGRA', 3), ('sayao', 'LOGRA', 1), ('440', 'LOGRA', 1), ('rod.br-010', '', 2), ('centro', '', 2), ('68675-000', '', 2), ('

Entities [('shcs qd.504 bl.b lj.55/56', 'LOGRA')]
Tokens [('shcs', 'LOGRA', 3), ('qd.504', 'LOGRA', 1), ('bl.b', 'LOGRA', 1), ('lj.55/56', 'LOGRA', 1), ('terreo', '', 2), ('e', '', 2), ('subsolo', '', 2), ('asa', '', 2), ('sul', '', 2), ('70331-525', '', 2), ('brasilia', '', 2), ('df', '', 2)]

Entities [('rua dr brandao 54', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('dr', 'LOGRA', 1), ('brandao', 'LOGRA', 1), ('54', 'LOGRA', 1), ('360305', '', 2), ('centro', '', 2), ('37400-000', '', 2), ('campanha', '', 2), ('mg', '', 2)]

Entities [('av.central 625/635', 'LOGRA')]
Tokens [('av.central', 'LOGRA', 3), ('625/635', 'LOGRA', 1), ('nuc', '', 2), ('bandeirante', '', 2), ('71710-520', '', 2), ('brasilia', '', 2), ('(', '', 2), ('nucleo', '', 2), ('bandeirante', '', 2), (')', '', 2), ('df', '', 2)]

Entities [('av. general canabarro 1103', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('general', 'LOGRA', 1), ('canabarro', 'LOGRA', 1), ('1103', 'LOGRA', 1), ('centro', '', 2), ('9901

...

Entities [('av. jo�o gomes vieira 28', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('jo', 'LOGRA', 1), ('�', 'LOGRA', 1), ('o', 'LOGRA', 1), ('gomes', 'LOGRA', 1), ('vieira', 'LOGRA', 1), ('28', 'LOGRA', 1), ('centro', '', 2), ('35277-000', '', 2), ('sao', '', 2), ('joao', '', 2), ('do', '', 2), ('manteninha', '', 2), ('mg', '', 2)]

Entities [('alameda joaquim eugenio de lima 133', 'LOGRA')]
Tokens [('alameda', 'LOGRA', 3), ('joaquim', 'LOGRA', 1), ('eugenio', 'LOGRA', 1), ('de', 'LOGRA', 1), ('lima', 'LOGRA', 1), ('133', 'LOGRA', 1), ('bela', '', 2), ('vista', '', 2), ('01403-001', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Entities [('av. vicente ferreira 539', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('vicente', 'LOGRA', 1), ('ferreira', 'LOGRA', 1), ('539', 'LOGRA', 1), ('centro', '', 2), ('75555-000', '', 2), ('vicentinopolis', '', 2), ('go', '', 2)]

Entities [('rua padre francisco', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('padre', 'LO

Entities [('r.da rodoviaria s/n', 'LOGRA')]
Tokens [('r.da', 'LOGRA', 3), ('rodoviaria', 'LOGRA', 1), ('s', 'LOGRA', 1), ('/', 'LOGRA', 1), ('n', 'LOGRA', 1), ('terminal', '', 2), ('rodoviario', '', 2), ('centro', '', 2), ('42800-400', '', 2), ('camacari', '', 2), ('ba', '', 2)]

Entities [('av.sao carlos 1040', 'LOGRA')]
Tokens [('av.sao', 'LOGRA', 3), ('carlos', 'LOGRA', 1), ('1040', 'LOGRA', 1), ('centro', '', 2), ('13560-010', '', 2), ('sao', '', 2), ('carlos', '', 2), ('sp', '', 2)]

Entities [('estrada da gavea 899 lj.103', 'LOGRA')]
Tokens [('estrada', 'LOGRA', 3), ('da', 'LOGRA', 1), ('gavea', 'LOGRA', 1), ('899', 'LOGRA', 1), ('lj.103', 'LOGRA', 1), ('sao', '', 2), ('conrado', '', 2), ('22619-900', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), ('rj', '', 2)]

Entities [('av.santa marina 2651', 'LOGRA')]
Tokens [('av.santa', 'LOGRA', 3), ('marina', 'LOGRA', 1), ('2651', 'LOGRA', 1), ('conjunto', '', 2), ('2', '', 2), ('vila', '', 2), ('albertina', '', 2), ('02732-0

Entities [('av. ver.dr.jose marques penteado 446', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('ver.dr.jose', 'LOGRA', 1), ('marques', 'LOGRA', 1), ('penteado', 'LOGRA', 1), ('446', 'LOGRA', 1), ('centro', '', 2), ('18560-000', '', 2), ('ipero', '', 2), ('sp', '', 2)]

Entities [('av portugal 2949', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('portugal', 'LOGRA', 1), ('2949', 'LOGRA', 1), ('sta', '', 2), ('amelia', '', 2), ('31555-000', '', 2), ('belo', '', 2), ('horizonte', '', 2), ('mg', '', 2)]

Entities [('rua dos andradas 449', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('dos', 'LOGRA', 1), ('andradas', 'LOGRA', 1), ('449', 'LOGRA', 1), ('centro', '', 2), ('97573-001', '', 2), ('santana', '', 2), ('do', '', 2), ('livramento', '', 2), ('rs', '', 2)]

Entities [('av.principal 479 cj.joao alves filho', 'LOGRA')]
Tokens [('av.principal', 'LOGRA', 3), ('479', 'LOGRA', 1), ('cj.joao', 'LOGRA', 1), ('alves', 'LOGRA', 1), ('filho', 'LOGRA', 1), ('taicoca', '', 2), ('49160-000', '', 2

Entities [('av.nove de abril 2246', 'LOGRA')]
Tokens [('av.nove', 'LOGRA', 3), ('de', 'LOGRA', 1), ('abril', 'LOGRA', 1), ('2246', 'LOGRA', 1), ('centro', '', 2), ('11510-003', '', 2), ('cubatao', '', 2), ('sp', '', 2)]

Entities [('av academia de sao paulo 310 e 312', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('academia', 'LOGRA', 1), ('de', 'LOGRA', 1), ('sao', 'LOGRA', 1), ('paulo', 'LOGRA', 1), ('310', 'LOGRA', 1), ('e', 'LOGRA', 1), ('312', 'LOGRA', 1), ('jardim', '', 2), ('camargo', '', 2), ('novo', '', 2), ('08121-400', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Entities [('r.duque de caxias 438', 'LOGRA')]
Tokens [('r.duque', 'LOGRA', 3), ('de', 'LOGRA', 1), ('caxias', 'LOGRA', 1), ('438', 'LOGRA', 1), ('centro', '', 2), ('19300-000', '', 2), ('presidente', '', 2), ('bernardes', '', 2), ('sp', '', 2)]

Entities [('rua prudente de morais', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('prudente', 'LOGRA', 1), ('de', 'LOGRA', 1), ('morais', 'LOGRA', 1), ('abadia', '', 2), ('

Entities [('av.sapopemba 2969', 'LOGRA')]
Tokens [('av.sapopemba', 'LOGRA', 3), ('2969', 'LOGRA', 1), ('v.regente', '', 2), ('feijo', '', 2), ('03345-001', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Entities [('rua floriano peixoto 131', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('floriano', 'LOGRA', 1), ('peixoto', 'LOGRA', 1), ('131', 'LOGRA', 1), ('sto', '', 2), ('antonio', '', 2), ('50020-060', '', 2), ('recife', '', 2), ('pe', '', 2)]

Entities [('av.vinte e cinco de abril 787', 'LOGRA')]
Tokens [('av.vinte', 'LOGRA', 3), ('e', 'LOGRA', 1), ('cinco', 'LOGRA', 1), ('de', 'LOGRA', 1), ('abril', 'LOGRA', 1), ('787', 'LOGRA', 1), ('sala', '', 2), ('1', '', 2), ('centro', '', 2), ('99370-000', '', 2), ('fontoura', '', 2), ('xavier', '', 2), ('rs', '', 2)]

Entities [('av. iguacu 2345', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('iguacu', 'LOGRA', 1), ('2345', 'LOGRA', 1), ('agua', '', 2), ('verde', '', 2), ('80240-030', '', 2), ('curitiba', '', 2), ('pr', '',

In [20]:
phrase = "SHVP Rua 12 435 casa 22 lote 1"

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('SHVP Rua 12 435', 'LOGRA')]
Tokens [('SHVP', 'LOGRA', 3), ('Rua', 'LOGRA', 1), ('12', 'LOGRA', 1), ('435', 'LOGRA', 1), ('casa', '', 2), ('22', '', 2), ('lote', '', 2), ('1', '', 2)]


In [ ]:
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

In [ ]:
loaded_model = spacy.load(output_dir)
evaluate(loaded_model, base_teste_final)